In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time
from pprint import pprint
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact
from tqdm.notebook import tqdm
import h5py

import nibabel as nib
from nilearn import surface
import bids
from bids import BIDSLayout

dir2 = os.path.abspath('../..')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)
    
from noise_ceiling import (
    compute_ncsnr,
    compute_nc,
)

from tc2see import load_data

In [2]:
tc2see_version = 3 # [1, 2]
dataset_root = Path('E:\\fmri_processing\\results')
dataset_path = dataset_root
derivatives_path = dataset_path / 'derivatives_TC2See'
data_path = derivatives_path / 'fmriprep'

# Initialize BIDSLayouts for querying files.
dataset_layout = BIDSLayout(dataset_path / 'TC2See')
derivatives_layout = BIDSLayout(derivatives_path / 'fmriprep', derivatives=True, validate = False)

c:\Users\James\anaconda3\envs\representationsLab\Lib\site-packages\bids\layout\layout.py:516: UserWarning: Derivative indexing was requested, but no valid datasets were found in the specified locations ([WindowsPath('E:/fmri_processing/results/derivatives_TC2See/fmriprep/derivatives')]). Note that all BIDS-Derivatives datasets must meet all the requirements for BIDS-Raw datasets (a common problem is to fail to include a 'dataset_description.json' file in derivatives datasets).
Example contents of 'dataset_description.json':
{"Name": "Example dataset", "BIDSVersion": "1.0.2", "GeneratedBy": [{"Name": "Example pipeline"}]}
  warnings.warn("Derivative indexing was requested, but no valid "


In [3]:
num_trs = 231 #231 #229 #236   # Total number of TRs in the fMRI data
tr = 2. # 1.97  # TR duration (in seconds)
mask_dilations = 3  # Number of dilation iterations for the brain mask
num_stimuli = 75 # 112  # Total number of different stimuli

num_runs = 6 if tc2see_version in (1, 3) else 8

subject_no = '07'
subjects = [subject_no]
task = "bird"
space = 'fsaverage' # ['T1w', 'MNI152NLin2009cAsym']

# Load stimulus images and create a mapping of stimulus names to unique identifiers
stimulus_images = h5py.File(derivatives_path / 'stimulus-images.hdf5', 'r')
stimulus_id_map = {name: i for i, name in enumerate(stimulus_images.attrs['stimulus_names'])}

new_or_append = 'w' # Use 'a' for append/overwrite, 'w' for new hdf5 file

for hemi in ('L', 'R'):
    with h5py.File(data_path / f'tc2see-v{tc2see_version}-fsaverage-surfs-{hemi}.hdf5', 'w') as f:
        for subject in tqdm(subjects):
            print(f"Processing subject {subject}...")
            leftOrRight = 0 if hemi == 'L' else 1
            group = f.require_group(f'sub-{subject}')
        
            fsaverage_surf = derivatives_layout.get(
                    subject=subject_no,
                    run=1,
                    task=task,
                    space=space, 
                    extension='func.gii',
            )[leftOrRight]

            fsaverage_surf = surface.load_surf_data(fsaverage_surf).astype(np.float64)

            num_voxels = fsaverage_surf.shape[0]
            print(f'{num_voxels=}')
                
            group.require_dataset('surf', shape=(num_runs, num_trs, num_voxels), dtype='f4')
            group.require_dataset('surf_mean', shape=(num_runs, num_voxels), dtype='f4')
            group.require_dataset('surf_std', shape=(num_runs, num_voxels), dtype='f4')
            group.require_dataset('surf_trend', shape=(num_runs, 2, num_voxels), dtype='f4')
            group.require_dataset('surf_trend_std', shape=(num_runs, num_voxels), dtype='f4')
            group.require_dataset('stimulus_trs', shape=(num_runs, num_stimuli), dtype='f4')
            group.require_dataset('stimulus_ids', shape=(num_runs, num_stimuli), dtype='i4')
            
            for run_id in tqdm(range(num_runs)):
                fsaverage_surf = derivatives_layout.get(
                    subject=subject_no,
                    run=run_id + 1,
                    task=task,
                    space=space, 
                    extension='func.gii',
                )[leftOrRight] # index 0 is left hemisphere data, 1 is right

                fsaverage_surf = np.transpose(surface.load_surf_data(fsaverage_surf).astype(np.float64))
                
                num_trs_run = fsaverage_surf.shape[0]
                print(f'{num_trs_run=}')
                trend_coeffs = np.stack([np.arange(num_trs_run), np.ones(shape=num_trs_run)], axis=1)
                
                # Perform linear detrending on the surf data
                surf_trend = np.linalg.lstsq(trend_coeffs, fsaverage_surf, rcond=None)[0]
                surf_predicted = trend_coeffs @ surf_trend
                surf_detrend = fsaverage_surf - surf_predicted

                # Load events data for the current subject and run
                events_file = dataset_layout.get(
                    subject=subject,
                    run=run_id + 1,
                    task=task,
                    extension='tsv'
                )[0]
                
                events_df = pd.read_csv(events_file.path, sep='\t')
                events_df = events_df[events_df['stimulus'] != '+']
                stimulus_names = [Path(stimulus_path).stem for stimulus_path in events_df['stimulus']]
                stimulus_names = [
                    name[:name.find('hash')-1] if "hash" in name else name
                    for name in stimulus_names
                ]
                stimulus_ids = [stimulus_id_map[name] for name in stimulus_names]
                
                stimulus_trs = np.array(events_df['tr']).astype(np.float32)
                
                # Store various datasets in the HDF5 file
                group['surf'][run_id, :num_trs_run] = fsaverage_surf
                group['surf_mean'][run_id] = fsaverage_surf.mean(axis=0)
                group['surf_std'][run_id] = fsaverage_surf.std(axis=0)
                group['surf_trend'][run_id] = surf_trend
                group['surf_trend_std'][run_id] = surf_detrend.std(axis=0)
                group['stimulus_trs'][run_id] = stimulus_trs
                group['stimulus_ids'][run_id] = stimulus_ids

  0%|          | 0/1 [00:00<?, ?it/s]

Processing subject 07...
num_voxels=163842


  0%|          | 0/6 [00:00<?, ?it/s]

num_trs_run=231
num_trs_run=231
num_trs_run=231
num_trs_run=231
num_trs_run=231
num_trs_run=231


In [10]:
for hemi in ('L', 'R'):
    bold, stimulus_ids = load_data(
        data_path / f'tc2see-v{tc2see_version}-fsaverage-surfs-{hemi}.hdf5', 
        f'sub-{subject_no}', 
        tr_offset=3,
        run_normalize='linear_trend',
        interpolation=False,
    )

    ncsnr = compute_ncsnr(bold, stimulus_ids) # Compute noise ceiling noise ratio
    nc = compute_nc(ncsnr, num_averages=1)

    img = nib.gifti.GiftiImage(darrays=[nib.gifti.GiftiDataArray(nc.astype(np.float32))])
    nib.save(img, f'E:/fmri_processing/results/visualization/fsaverage_surface_nc/sub{subject_no}_fsaverage_surf_nc_{hemi}.gii')

C:\Users\James\AppData\Local\Temp\ipykernel_1872\3575593899.py:66: RuntimeWarning: invalid value encountered in divide
  run_bold = (run_bold - predicted_bold) / group['bold_trend_std'][i]


(450, 126733)
[-1.2188748  -1.1817868   0.46749604 ...  0.848829    0.82020146
 -0.6242765 ]
(450,)
